# VacationPY 

In [1]:
# Dependencies and Setup 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
import requests 
import gmaps 
import os 

# API key 
from api_keys2 import g_key

# Store Part I results into DataFrame

In [2]:
# Output File (CSV)
part1_results_path = "../Output_Data/cities_copy.csv"

# Read Data
part1_results = pd.read_csv(part1_results_path)

part1_results

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Abu Samrah,35.30,37.18,62.74,90,0,7.34,SY,1602799402
1,Ifo,6.81,3.20,78.80,94,20,3.36,NG,1602799403
2,Abu Dhabi,24.47,54.37,82.40,48,0,2.24,AE,1602799205
3,Port Blair,11.67,92.75,80.56,83,99,12.24,IN,1602799403
4,Jieshou,33.26,115.36,56.16,94,100,3.02,CN,1602799403
...,...,...,...,...,...,...,...,...,...
537,Disna,55.57,28.20,42.44,95,21,3.62,BY,1602799544
538,Bell Ville,-32.63,-62.69,77.00,41,22,1.99,AR,1602799544
539,Sabang,5.89,95.32,82.29,78,99,20.89,ID,1602799424
540,Port Macquarie,-31.43,152.92,73.99,68,15,6.93,AU,1602799372


# Humidity Heatmap 

In [3]:
# Configure GMaps 
gmaps.configure(api_key = g_key)

# Use Latitude and Longitufe as locations 
coordinates = part1_results[["Lat", "Lng"]]

# Plot Heatmap 
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(part1_results["Humidity"])

# Create heat layer 
heat_layer = gmaps.heatmap_layer(coordinates, weights = part1_results["Humidity"], 
                                 dissipating=False, max_intensity=100, point_radius=3)

# Add Layer 
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
max_intensity

100

# Create new DataFrame fitting weather criteria 

In [5]:
# Narrow down the cities to fit weather conditions 
narrowed_city_df = part1_results.loc[(part1_results["Max Temp"] < 80) & 
                                    (part1_results["Max Temp"] > 70) &
                                   (part1_results["Wind Speed"] < 10) &
                                   (part1_results["Cloudiness"] == 0)].dropna()

narrowed_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,Islāmpur,26.27,88.20,76.37,76,0,3.15,IN,1602799406
75,Vicuña,-30.03,-70.71,73.99,30,0,4.00,CL,1602799490
89,Cap Malheureux,-19.98,57.61,73.99,75,0,4.00,MU,1602799491
112,Mufumbwe,-13.68,24.80,70.99,58,0,6.15,ZM,1602799494
132,Buraidah,26.33,43.98,75.20,27,0,2.24,SA,1602799475
246,Hargeisa,9.56,44.06,70.75,48,0,8.39,SO,1602799511
282,Tezu,27.92,96.17,75.25,56,0,2.80,IN,1602799515
288,Tsabong,-26.02,22.40,79.00,19,0,9.98,BW,1602799516
303,Jaleshwar,26.65,85.80,76.14,80,0,1.45,NP,1602799518
363,Ballina,-28.87,153.57,71.01,82,0,2.24,AU,1602799524


# Hotel Map 

In [6]:
# Create Hotel DataFrame
hotel_df = narrowed_city_df.loc[:, ["City", "Country", "Lat", "Lng"]]

# Add a "Hotel Name" column 
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
28,Islāmpur,IN,26.27,88.20,
75,Vicuña,CL,-30.03,-70.71,
89,Cap Malheureux,MU,-19.98,57.61,
112,Mufumbwe,ZM,-13.68,24.80,
132,Buraidah,SA,26.33,43.98,
246,Hargeisa,SO,9.56,44.06,
282,Tezu,IN,27.92,96.17,
288,Tsabong,BW,-26.02,22.40,
303,Jaleshwar,NP,26.65,85.80,
363,Ballina,AU,-28.87,153.57,


In [7]:
# Parameters Dictionary to update each iteration 
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    # Get Latitude and Longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params = params).json()
    
    try: 
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except:
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.


In [8]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
28,Islāmpur,IN,26.27,88.20,Hotel Saluja
75,Vicuña,CL,-30.03,-70.71,CasaMolle
89,Cap Malheureux,MU,-19.98,57.61,O'Biches by Horizon Holidays
112,Mufumbwe,ZM,-13.68,24.80,
132,Buraidah,SA,26.33,43.98,Swiss International Resort Unaizah - Al Qassim
246,Hargeisa,SO,9.56,44.06,HYATT HOTEL
282,Tezu,IN,27.92,96.17,Dibang Valley Jungle Camp
288,Tsabong,BW,-26.02,22.40,Dikukama Hotel
303,Jaleshwar,NP,26.65,85.80,Hotel Sita Sharan
363,Ballina,AU,-28.87,153.57,Ramada Hotel & Suites by Wyndham Ballina Byron


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map 
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map 
fig.add_layer(markers)

# Diplay Map 
fig 

Figure(layout=FigureLayout(height='420px'))